In [ ]:
# Connect to Snowflake and write the data to a table
from snowflake.snowpark.session import Session

connection_params = {
  "account": "<your_account>",
  "user": "<your_username>",
  "password": "<your_password>",
  "role": "ACCOUNTADMIN",
  "warehouse": "COMPUTE_WH",
  "database": "AVALANCHE_DB",
  "schema": "AVALANCHE_SCHEMA"
}

session = Session.builder.configs(connection_params).create()

query = """
SELECT
    PRODUCT,
    CUSTOMER_REVIEW,
    SNOWFLAKE.CORTEX.SENTIMENT(CUSTOMER_REVIEW) as SENTIMENT_SCORE
FROM 
    PARSED_REVIEWS
"""

df_reviews = session.sql(query).to_pandas()

df_reviews.head()

In [ ]:
import matplotlib.pyplot as plt

product_sentiment = df_reviews.groupby("PRODUCT")["SENTIMENT_SCORE"].mean().sort_values()
product_sentiment.plot(kind="barh", title="Average Sentiment by Product")
plt.xlabel("Sentiment Score")
plt.tight_layout()
plt.show()

In [ ]:
query = """
SELECT 
    "Shipping_Date",
    COUNT(*) AS SHIPMENT_COUNT
FROM 
    SHIPPING_LOGS
GROUP BY
    "Shipping_Date"
ORDER BY 
    "Shipping_Date"
"""

shipment_df = session.sql(query).to_pandas()

shipment_df.head()

In [ ]:
shipment_df.plot(x="Shipping_Date", y="SHIPMENT_COUNT", kind="line", title="Shipments Per Day")
plt.ylabel("Number of Shipments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
low_volume_days = shipment_df[shipment_df["SHIPMENT_COUNT"] < 5]
print("Low-volume shipping days:\n", low_volume_days)

In [ ]:
query = """
SELECT
    *
FROM 
    SHIPPING_LOGS
"""

df_shipping = session.sql(query).to_pandas()

df_shipping.head()

In [ ]:
carrier_counts = df_shipping.groupby("Carrier")["Order_ID"].count()
carrier_counts.plot(kind="bar", title="Total Shipments by Carrier")
plt.ylabel("Number of Shipments")
plt.tight_layout()
plt.show()